# Players Playing Today

This is scraped from rotowire

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests   
import shutil      
import datetime
from scipy.stats import norm
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
import os
import winsound
os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1')
import warnings
warnings.filterwarnings('ignore')
import re


from selenium.common.exceptions import WebDriverException


### Attempt

In [2]:
driver = webdriver.Chrome()
url = 'https://www.rotowire.com/basketball/nba-lineups.php'

In [4]:


driver.get(url)

soup = BeautifulSoup(driver.page_source, 'html.parser')

# find div class = 'lineups'. This is the div that contains all the lineups.
# The lineups are split by 'lineup is-nba' class

tot_lineup_df = pd.DataFrame()

lineups = soup.find_all('div', class_='lineup__box')
# get rid of last 2 lineups
lineups = lineups[:-2]

for lineup in lineups:
    # wait for page to load
    time.sleep(2)

    away_team = lineup.find('ul', class_= 'lineup__list is-visit').text.strip().replace('\n', ' ')
    home_team = lineup.find('ul', class_= 'lineup__list is-home').text.strip().replace('\n', ' ')

         # split into three lists by 'Expected Lineup'
    away_team_name = lineup.find('a', class_='lineup__team is-visit').text.strip()
    home_team_name = lineup.find('a', class_='lineup__team is-home').text.strip()

    away_team_pg = away_team[away_team.find('PG')+3:away_team.find('SG')-1]
    away_team_sg = away_team[away_team.find('SG')+3:away_team.find('SF')-1]
    away_team_sf = away_team[away_team.find('SF')+3:away_team.find('PF')-1]
    away_team_pf = away_team[away_team.find('PF')+3:away_team.find(' C ')]
    away_team_c = away_team[away_team.find(' C ')+2:away_team.find('Pro')-1]

    home_team_pg = home_team[home_team.find('PG')+3:home_team.find('SG')-1]
    home_team_sg = home_team[home_team.find('SG')+3:home_team.find('SF')-1]
    home_team_sf = home_team[home_team.find('SF')+3:home_team.find('PF')-1]
    home_team_pf = home_team[home_team.find('PF')+3:home_team.find(' C ')]
    home_team_c = home_team[home_team.find(' C ')+2:home_team.find('Pro')-1]

    away_status = away_team[0:away_team.find('PG')-2]
    home_status = home_team[0:home_team.find('PG')-2]

    # create dataframe
    lineup_df = pd.DataFrame({'Away_Team': away_team_name, 'Home_Team': home_team_name, 'Away_PG': away_team_pg, 'Away_SG': away_team_sg, 'Away_SF': away_team_sf, 'Away_PF': away_team_pf, 'Away_C': away_team_c, 'Home_PG': home_team_pg, 'Home_SG': home_team_sg, 'Home_SF': home_team_sf, 'Home_PF': home_team_pf, 'Home_C': home_team_c, 'Away_Status': away_status, 'Home_Status': home_status}, index=[0])

    tot_lineup_df = pd.concat([tot_lineup_df, lineup_df], axis=0)

tot_lineup_df = tot_lineup_df.reset_index(drop=True)

tot_lineup_df

,Away_Team,Home_Team,Away_PG,Away_SG,Away_SF,Away_PF,Away_C,Home_PG,Home_SG,Home_SF,Home_PF,Home_C,Away_Status,Home_Status
0,TOR,MIL,Fred VanVleet,Gary Trent,OG Anunoby,S. Barnes,Pascal Siakam,Jrue Holiday,Grayson Allen,P. Connaughton,Bobby Portis,Brook Lopez,Expected Lineup,Expected Lineup
1,BKN,SAS,K. Irving GTD,Joe Harris,Royce O'Neale,Ben Simmons,Nic Claxton,Tre Jones,R. Langford,K. Johnson,Jeremy Sochan,Jakob Poeltl,Expected Lineup,Expected Lineup
2,POR,DEN,D. Lillard,A. Simons,Josh Hart,Jerami Grant,Jusuf Nurkic,J. Murray GTD,K. Caldwell-Pope,M. Porter,Aaron Gordon,Nikola Jokic,Expected Lineup,Expected Lineup
3,PHI,LAC,James Harden,D. Melton GTD,Tobias Harris,P.J. Tucker,Joel Embiid,Terance Mann,P. George GTD,Kawhi Leonard,Marcus Morris,Ivica Zubac,Expected Lineup,Expected Lineup


In [ ]:
today = datetime.datetime.now().strftime('%Y-%m-%d')

tot_lineup_df.to_csv('data/team/aggregates/daily_updates/starting_lineups_{}.csv'.format(today), index=False)

In [3]:
# driver.get(url)
# # find div class = 'lineups'
# lineups = driver.find_element_by_class_name('lineups')
# # find div class lineup__main within lineups, and get text
# team_names = lineups.find_elements_by_class_name('lineup__top')
# lineup_main = lineups.find_elements_by_class_name('lineup__main')

# games_tonight = pd.DataFrame()

# n=1

# for game in team_names:
#     # get text from game
#     game_text = game.text
#     away_team = game_text[:3]
#     home_team = game_text[-3:]
#     matchup = away_team + ' @ ' + home_team
#     game = 'Game_' + str(n)
#     # append
#     games_tonight = games_tonight.append({'away_team': away_team, 'home_team': home_team, 'matchup' : matchup, 'Game' : game}, ignore_index=True)
#     n+=1

# # last two rows are not needed
# games_tonight = games_tonight[:-2]
# games_tonight

,away_team,home_team,matchup,Game
0,DET,PHI,DET @ PHI,Game_1
1,OKC,MIA,OKC @ MIA,Game_2
2,CHA,TOR,CHA @ TOR,Game_3
3,CLE,UTA,CLE @ UTA,Game_4
4,PHX,GSW,PHX @ GSW,Game_5
5,ORL,POR,ORL @ POR,Game_6
6,DAL,LAC,DAL @ LAC,Game_7


In [4]:
# # transpose games_tonight and make Game column the index
# games_tonight = games_tonight.T
# games_tonight.columns = games_tonight.iloc[3]

In [5]:
# url = 'https://www.rotowire.com/basketball/nba-lineups.php'
# driver.get(url)
# soup = BeautifulSoup(driver.page_source, 'html.parser')
# lineups = soup.find('div', class_='lineup__box')

# # get innerhtml of lineups div
# lineup_1 = lineups.find('div', class_='lineup__main')
# matchup_1 = lineups.find('div', class_='lineup__matchup').text.replace('\n', ' ').strip().replace('   ', '')
# team1 = matchup_1.split('  ')[0]
# team_1_record = matchup_1.split('  ')[1]
# team2 = matchup_1.split('  ')[2]
# team_2_record = matchup_1.split('  ')[3]

# team1, team_1_record, team2, team_2_record

('Pistons', '(11-32)', '76ers', '(24-15)')

In [6]:
# # for each game in games_tonight, create two columns, one for each team

# games_by_team = pd.DataFrame()

# for game in games_tonight.columns:
#     away_team = games_tonight[game]['away_team']
#     home_team = games_tonight[game]['home_team']
#     games_by_team[str(game) +'_'+ away_team] = np.nan
#     games_by_team[str(game) +'_'+ home_team] = np.nan

# games_by_team

,Game_1_DET,Game_1_PHI,Game_2_OKC,Game_2_MIA,Game_3_CHA,Game_3_TOR,Game_4_CLE,Game_4_UTA,Game_5_PHX,Game_5_GSW,Game_6_ORL,Game_6_POR,Game_7_DAL,Game_7_LAC


In [7]:
# # get visitor names and values
# visitor = lineup_1.find('ul').text.replace('\n', '  ').strip().replace('   ', ' ')
# visitor = visitor.replace('PG  ', 'PG ').replace('SG  ', 'SG ').replace('SF  ', 'SF ').replace('PF  ', 'PF ').replace('C  ', 'C ').replace('F  ','F ').replace('G  ', 'G ').replace('  GTD', ' GTD').replace('  OUT', ' OUT')
# # split into list by '  '
# visitor = visitor.split('  ')
# visitor = [x for x in visitor if x != '']
# visitor = [x.strip() for x in visitor]
# visitor = visitor[:6]
# visitor

['Confirmed Lineup',
 'PG Killian Hayes',
 'SG Jaden Ivey',
 'SF Saddiq Bey',
 'PF Kevin Knox',
 'C Nerlens Noel']

In [8]:
# home = lineup_1.find('ul').next_sibling.next_sibling.text.replace('\n', '  ').strip().replace('   ', ' ')
# home = home.replace('PG  ', 'PG ').replace('SG  ', 'SG ').replace('SF  ', 'SF ').replace('PF  ', 'PF ').replace('C  ', 'C ').replace('F  ','F ').replace('G  ', 'G ').replace('  GTD', ' GTD').replace('  OUT', ' OUT')
# home = home.split('  ')
# home = [x for x in home if x != '']
# home = [x.strip() for x in home]
# home = home[:6]
# home

['Confirmed Lineup',
 'PG James Harden',
 'SG Tyrese Maxey',
 'SF Tobias Harris',
 'PF P.J. Tucker',
 'C Joel Embiid']

### Attempt 3 (good one?)

In [4]:
# # all together?
# import re
# driver.get(url)
# # find div class = 'lineups'. This is the div that contains all the lineups.
# # The lineups are split by 'lineup is-nba' class

# tot_lineup_df = pd.DataFrame()

# lineups = driver.find_elements_by_class_name('lineup__box')
# for lineup in lineups:
#     # wait for page to load
#     time.sleep(2)
#     lineup_text = lineup.text.replace('\n', ' ').strip()
#         # split into three lists by 'Expected Lineup'

#     if 'Expected Lineup' in lineup_text:
#         lineup_text = lineup_text.split('Expected Lineup')
#         lineup_status = 'Expected Lineup'
#     elif 'Confirmed Lineup' in lineup_text:
#         lineup_text = lineup_text.split('Confirmed Lineup')
#         lineup_status = 'Confirmed Lineup'

#     lineup_text1 = lineup_text[0]
#     lineup_text2 = lineup_text[1]
#     lineup_text3 = lineup_text[2]

#     # split lineup_text3 into two lists by 'Referees'
#     lineup_text3 = lineup_text3.split('Referees')[0]

#     lineup_text1, lineup_text2, lineup_text3

#     away_team = lineup_text1[:3]
#     home_team = lineup_text1[4:7]
#     # away team players df


#     away_team_pg = lineup_text2[lineup_text2.find('PG')+3:lineup_text2.find('SG')-1]
#     away_team_sg = lineup_text2[lineup_text2.find('SG')+3:lineup_text2.find('SF')-1]
#     away_team_sf = lineup_text2[lineup_text2.find('SF')+3:lineup_text2.find('PF')-1]
#     away_team_pf = lineup_text2[lineup_text2.find('PF')+3:lineup_text2.find(' C ')]
#     away_team_c = lineup_text2[lineup_text2.find(' C ')+2:lineup_text2.find('Pro')-1]


#     # home team players df
#     home_team_pg = lineup_text3[lineup_text3.find('PG')+3:lineup_text3.find('SG')-1]
#     home_team_sg = lineup_text3[lineup_text3.find('SG')+3:lineup_text3.find('SF')-1]
#     home_team_sf = lineup_text3[lineup_text3.find('SF')+3:lineup_text3.find('PF')-1]
#     home_team_pf = lineup_text3[lineup_text3.find('PF')+3:lineup_text3.find(' C ')]
#     home_team_c = lineup_text3[lineup_text3.find(' C ')+2:lineup_text3.find('Pro')-1]


#     # Create df with all info
#     away_df = pd.DataFrame({'Team': away_team, 'Status': lineup_status,'PG': away_team_pg, 'SG': away_team_sg, 'SF': away_team_sf, 'PF': away_team_pf, 'C': away_team_c}, index=[0])

#     home_df = pd.DataFrame({'Team': home_team, 'Status': lineup_status,'PG': home_team_pg, 'SG': home_team_sg, 'SF': home_team_sf, 'PF': home_team_pf, 'C': home_team_c}, index=[0])

#     # Concatenate dfs
#     lineup_df = pd.concat([away_df, home_df], axis=0)
 

#     # append to tot_lineup_df
#     tot_lineup_df = tot_lineup_df.append(lineup_df)

#     # reset index
#     tot_lineup_df = tot_lineup_df.reset_index(drop=True)

# tot_lineup_df = tot_lineup_df[:-4]
# tot_lineup_df

IndexError: list index out of range

In [5]:
# driver.get(url)
# # find div class = 'lineups'. This is the div that contains all the lineups.
# # The lineups are split by 'lineup is-nba' class

# all_lineups = driver.find_element_by_class_name('lineups')
# # find div class 'lineup__box' within lineups, and get text
# lineup = driver.find_element_by_class_name('lineup__box')
# lineup_text = lineup.text.replace('\n', ' ').strip()
#     # split into three lists by 'Expected Lineup'

# if 'Expected Lineup' in lineup_text:
#     lineup_text = lineup_text.split('Expected Lineup')
#     lineup_status = 'Expected Lineup'
# elif 'Confirmed Lineup' in lineup_text:
#     lineup_text = lineup_text.split('Confirmed Lineup')
#     lineup_status = 'Confirmed Lineup'

# lineup_text1 = lineup_text[0]
# lineup_text2 = lineup_text[1]
# lineup_text3 = lineup_text[2]

# # split lineup_text3 into two lists by 'Referees'
# lineup_text3 = lineup_text3.split('Referees')[0]

# lineup_text1, lineup_text2, lineup_text3

('DET PHI Pistons (11-32) 76ers (24-15) ',
 ' PG Killian Hayes SG Jaden Ivey SF Saddiq Bey PF Kevin Knox C Nerlens Noel Projected Minutes On/Off Court Stats MAY NOT PLAY C M. Bagley OUT F B. Bogdanovic OUT C J. Duren OUT F I. Livers OUT C I. Stewart OUT ',
 ' PG James Harden SG Tyrese Maxey SF Tobias Harris PF P.J. Tucker C Joel Embiid Projected Minutes On/Off Court Stats MAY NOT PLAY None ')

In [11]:
# import re

# away_team = lineup_text1[:3]
# home_team = lineup_text1[4:7]
# # away team players df
# # get lineup_text1.find('PG')+3 lineup_text1.find('SG')-1

# away_team_pg = lineup_text2[lineup_text2.find('PG')+3:lineup_text2.find('SG')-1]
# away_team_sg = lineup_text2[lineup_text2.find('SG')+3:lineup_text2.find('SF')-1]
# away_team_sf = lineup_text2[lineup_text2.find('SF')+3:lineup_text2.find('PF')-1]
# away_team_pf = lineup_text2[lineup_text2.find('PF')+3:lineup_text2.find('C')-1]
# away_team_c = lineup_text2[lineup_text2.find('C')+2:lineup_text2.find('Pro')-1]

# #add may not play
# away_mnp_1 = lineup_text2.split('MAY NOT PLAY')[1]
# # split at 'GTD' and 'OUT' 
# away_mnp_1 = re.split('GTD|OUT', away_mnp_1)

# away_not_playing = []

# for player in away_mnp_1:
#     # if line is not empty
#     if len(player) > 2:
#         # add new variable for each player to store the player name
#         player = player.strip()
#         # add player to list
#         away_not_playing.append(player)

# try:
#     away_not_playing_1 = away_not_playing[0]
# except:
#     away_not_playing_1 = ''

# try:
#     away_not_playing_2 = away_not_playing[1]
# except:
#     away_not_playing_2 = ''

# try:
#     away_not_playing_3 = away_not_playing[2]
# except:
#     away_not_playing_3 = ''

# try:
#     away_not_playing_4 = away_not_playing[3]
# except:
#     away_not_playing_4 = ''

# try:
#     away_not_playing_5 = away_not_playing[4]
# except:
#     away_not_playing_5 = ''

# try:
#     away_not_playing_6 = away_not_playing[5]
# except:
#     away_not_playing_6 = ''

# try:
#     away_not_playing_7 = away_not_playing[6]
# except:
#     away_not_playing_7 = ''

# try:
#     away_not_playing_8 = away_not_playing[7]
# except:
#     away_not_playing_8 = ''

# try:
#     away_not_playing_9 = away_not_playing[8]
# except:
#     away_not_playing_9 = ''

# try:
#     away_not_playing_10 = away_not_playing[9]
# except:
#     away_not_playing_10 = ''


# away_mnp_2 = lineup_text3.split('MAY NOT PLAY')[1]

# # home team players df
# home_team_pg = lineup_text3[lineup_text3.find('PG')+3:lineup_text3.find('SG')-1]
# home_team_sg = lineup_text3[lineup_text3.find('SG')+3:lineup_text3.find('SF')-1]
# home_team_sf = lineup_text3[lineup_text3.find('SF')+3:lineup_text3.find('PF')-1]
# home_team_pf = lineup_text3[lineup_text3.find('PF')+3:lineup_text3.find('C')-1]
# home_team_c = lineup_text3[lineup_text3.find('C')+2:lineup_text3.find('Pro')-1]

# away_mnp_2 = re.split('GTD|OUT', away_mnp_2)

# home_not_playing = []

# for player in away_mnp_2:
#     # if line is not empty
#     if len(player) > 2:
#         # add new variable for each player to store the player name
#         player = player.strip()
#         # add player to list
#         home_not_playing.append(player)

# try:
#     home_not_playing_1 = home_not_playing[0]
# except:
#     home_not_playing_1 = ''

# try:
#     home_not_playing_2 = home_not_playing[1]
# except:
#     home_not_playing_2 = ''

# try:
#     home_not_playing_3 = home_not_playing[2]
# except:
#     home_not_playing_3 = ''

# try:
#     home_not_playing_4 = home_not_playing[3]
# except:
#     home_not_playing_4 = ''

# try:
#     home_not_playing_5 = home_not_playing[4]
# except:
#     home_not_playing_5 = ''

# try:
#     home_not_playing_6 = home_not_playing[5]
# except:
#     home_not_playing_6 = ''

# try:
#     home_not_playing_7 = home_not_playing[6]
# except:
#     home_not_playing_7 = ''

# try:
#     home_not_playing_8 = home_not_playing[7]
# except:
#     home_not_playing_8 = ''

# try:
#     home_not_playing_9 = home_not_playing[8]
# except:
#     home_not_playing_9 = ''

# try:
#     home_not_playing_10 = home_not_playing[9]
# except:
#     home_not_playing_10 = ''

# # Create df with all info
# away_df = pd.DataFrame({'Team': away_team, 'Status': lineup_status,'PG': away_team_pg, 'SG': away_team_sg, 'SF': away_team_sf, 'PF': away_team_pf, 'C': away_team_c, 'MNP_1': away_not_playing_1, 'MNP_2': away_not_playing_2, 'MNP_3': away_not_playing_3, 'MNP_4': away_not_playing_4, 'MNP_5': away_not_playing_5, 'MNP_6': away_not_playing_6, 'MNP_7': away_not_playing_7, 'MNP_8': away_not_playing_8, 'MNP_9': away_not_playing_9, 'MNP_10': away_not_playing_10}, index=[0])

# home_df = pd.DataFrame({'Team': home_team, 'Status': lineup_status,'PG': home_team_pg, 'SG': home_team_sg, 'SF': home_team_sf, 'PF': home_team_pf, 'C': home_team_c, 'MNP_1': home_not_playing_1, 'MNP_2': home_not_playing_2, 'MNP_3': home_not_playing_3, 'MNP_4': home_not_playing_4, 'MNP_5': home_not_playing_5, 'MNP_6': home_not_playing_6, 'MNP_7': home_not_playing_7, 'MNP_8': home_not_playing_8, 'MNP_9': home_not_playing_9, 'MNP_10': home_not_playing_10}, index=[0])

# # Concatenate dfs
# lineup_df = pd.concat([away_df, home_df], axis=0)


# # delete first 2 characters in all MNP columns
# lineup_df['MNP_1'] = lineup_df['MNP_1'].str[2:]
# lineup_df['MNP_2'] = lineup_df['MNP_2'].str[2:]
# lineup_df['MNP_3'] = lineup_df['MNP_3'].str[2:]
# lineup_df['MNP_4'] = lineup_df['MNP_4'].str[2:]
# lineup_df['MNP_5'] = lineup_df['MNP_5'].str[2:]
# lineup_df['MNP_6'] = lineup_df['MNP_6'].str[2:]
# lineup_df['MNP_7'] = lineup_df['MNP_7'].str[2:]
# lineup_df['MNP_8'] = lineup_df['MNP_8'].str[2:]
# lineup_df['MNP_9'] = lineup_df['MNP_9'].str[2:]
# lineup_df['MNP_10'] = lineup_df['MNP_10'].str[2:]

# lineup_df

,Team,Status,PG,SG,SF,PF,C,MNP_1,MNP_2,MNP_3,MNP_4,MNP_5,MNP_6,MNP_7,MNP_8,MNP_9,MNP_10
0,DET,Confirmed Lineup,Killian Hayes,Jaden Ivey,Saddiq Bey,Kevin Knox,Nerlens Noel,M. Bagley,B. Bogdanovic,J. Duren,I. Livers,I. Stewart,,,,,
0,PHI,Confirmed Lineup,James Harden,Tyrese Maxey,Tobias Harris,P.J. Tucker,Joel Embiid,ne,,,,,,,,,


### Attempt 4 (Working)

In [12]:
# driver = webdriver.Chrome()

In [13]:
# driver.get(url)
# # find div class = 'lineups'. This is the div that contains all the lineups.
# # The lineups are split by 'lineup is-nba' class

# all_lineups = driver.find_elements_by_class_name('lineups')
# # find div class 'lineup__box' within lineups, and get text
# lineups = driver.find_elements_by_class_name('lineup__box')
# all_lineups_df = pd.DataFrame()

# num_lineups = (len(lineups))-2
# n = 1

# lineups_today = pd.DataFrame

# for l in lineups:
#     if n <= num_lineups:
#         #lineup = driver.find_element_by_class_name('lineup__box')
#         lineup_text = l.text.replace('\n', ' ').strip()
#         if 'Expected Lineup' in lineup_text:
#             lineup_text = lineup_text.split('Expected Lineup')
#         elif 'Confirmed Lineup' in lineup_text:
#             lineup_text = lineup_text.split('Confirmed Lineup')
        
#         matchup_text = lineup_text[0]
#         away_lineup = lineup_text[1]
#         away_may_not_play = away_lineup.split('MAY NOT PLAY')[1]
#         away_starters = away_lineup.split('MAY NOT PLAY')[0]
#         away_starters = away_starters.split('Projected')[0]
#         away_may_not_play = away_may_not_play.split('Referees')[0]
#         home_lineup = lineup_text[2]
#         home_may_not_play = home_lineup.split('MAY NOT PLAY')[1]
#         home_starters = home_lineup.split('MAY NOT PLAY')[0]
#         home_starters = home_starters.split('Projected')[0]
#         home_may_not_play = home_may_not_play.split('Referees')[0]
#         # append to lineups_today
#         lineups_today = pd.DataFrame({'Matchup': matchup_text, 'Away Starters': away_starters, 'Away May Not Play': away_may_not_play, 'Home Starters': home_starters, 'Home May Not Play': home_may_not_play}, index=[0])
#         all_lineups_df = all_lineups_df.append(lineups_today)

#         n += 1
#     else:
#         pass

# all_lineups_df = all_lineups_df.reset_index(drop=True)
# all_lineups_df

,Matchup,Away Starters,Away May Not Play,Home Starters,Home May Not Play
0,DET PHI Pistons (11-32) 76ers (24-15),PG Killian Hayes SG Jaden Ivey SF Saddiq Bey ...,C M. Bagley OUT F B. Bogdanovic OUT C J. Dure...,PG James Harden SG Tyrese Maxey SF Tobias Har...,None
1,OKC MIA Thunder (18-22) Heat (21-20),PG S. Gilgeous-Alexander SG Josh Giddey SF Lu...,F O. Dieng OUT F A. Pokusevski OUT C J. Robin...,PG V. Oladipo SG Max Strus SF Jimmy Butler PF...,C U. Haslem GTD C B. Adebayo OUT G T. Herro O...
2,CHA TOR Hornets (11-30) Raptors (17-23),PG LaMelo Ball SG Terry Rozier SF J. McDaniel...,F G. Hayward OUT F K. Oubre OUT,PG Fred VanVleet SG Gary Trent SF OG Anunoby ...,
3,CLE UTA Cavaliers (26-15) Jazz (20-23),PG D. Garland SG D. Mitchell SF Isaac Okoro P...,G R. Rubio OUT F Dean Wade OUT G D. Windler OUT,PG Mike Conley SG J. Clarkson SF L. Markkanen...,C U. Azubuike OUT G L. Bolmaro OUT F T. Horto...
4,PHX GSW Suns (20-21) Warriors (20-20),PG L. Shamet GTD SG Mikal Bridges SF T. Craig...,F T. Craig GTD G L. Shamet GTD C D. Ayton OUT...,PG Stephen Curry SG K. Thompson GTD SF A. Wig...,F A. Iguodala GTD G K. Thompson GTD F J. Gree...
5,ORL POR Magic (15-26) Trail Blazers (19-20),PG M. Fultz SG Gary Harris SF Franz Wagner PF...,F J. Isaac OUT F C. Okeke OUT,PG D. Lillard SG A. Simons SF Josh Hart PF Je...,C J. Nurkic GTD G G. Payton GTD F N. Little O...
6,DAL LAC Mavericks (23-18) Clippers (21-21),PG L. Doncic GTD SG S. Dinwiddie SF Tim Harda...,G L. Doncic GTD F D. Finney-Smith OUT F Josh ...,PG Terance Mann SG Nicolas Batum SF Kawhi Leo...,G P. George OUT G L. Kennard OUT


In [14]:
# def get_starters(index_num):
#     data = all_lineups_df[index_num:index_num+1]
#     home_team = data['Matchup'].str[4:7]
#     away_team = data['Matchup'].str[0:3]

#     # away_pg is the words between PG and SG
#     away_pg = data['Away Starters'].str.findall(r'PG(.*?)SG').str[0]
#     # away_sg is the words between SG and SF
#     away_sg = data['Away Starters'].str.findall(r'SG(.*?)SF').str[0]
#     # away_sf is the words between SF and PF
#     away_sf = data['Away Starters'].str.findall(r'SF(.*?)PF').str[0]
#     # away_pf is the words between PF and C
#     away_pf = data['Away Starters'].str.findall(r'PF(.*?)C').str[0]
#     # away_c is the words after C
#     away_c = data['Away Starters'].str.findall(r'C(.*)').str[0]
    
#     # home_pg is the words between PG and SG
#     home_pg = data['Home Starters'].str.findall(r'PG(.*?)SG').str[0]
#     # home_sg is the words between SG and SF
#     home_sg = data['Home Starters'].str.findall(r'SG(.*?)SF').str[0]
#     # home_sf is the words between SF and PF
#     home_sf = data['Home Starters'].str.findall(r'SF(.*?)PF').str[0]
#     # home_pf is the words between PF and C
#     home_pf = data['Home Starters'].str.findall(r'PF(.*?)C').str[0]
#     # home_c is the words after C
#     home_c = data['Home Starters'].str.findall(r'C(.*)').str[0]

#     starters = pd.DataFrame({'Away_Team' : away_team, 'Home_Team' : home_team, 'Away PG': away_pg, 'Away SG': away_sg, 'Away SF': away_sf, 'Away PF': away_pf, 'Away C': away_c, 'Home PG': home_pg, 'Home SG': home_sg, 'Home SF': home_sf, 'Home PF': home_pf, 'Home C': home_c})


#     return starters